# Curso: "El lenguaje de programación Python para la docencia en el ámbito científico"

&nbsp;  
<img src="../images/logo_python_letras.png" align="center" style="width:350px;"/>

<strong><div style="text-align: center"> [Mabel Delgado Babiano](https://es.linkedin.com/in/mabeldelgadob)</div></strong>

&nbsp;  
<div style="text-align: center">Heredia, Costa Rica</div>
<div style="text-align: center">4 - 7 Febrero 2019</div>


# AeroPython: Salto de Felix Baumgartner desde la estratosfera

El 14 de octubre de 2012 Felix Baumgartner saltó de una sonda estratosférica a casi 40000 metros, batiendo los récords de vuelo en globo tripulado a mayor altura y salto a mayor altura. **¿Rompió además la barrera del sonido?**
¡**Python**! ;)

<img src="http://undondin.com/wp-content/uploads/2013/01/red-bull-stratos.jpg" width="800" />

Inspiración: http://pybonacci.wordpress.com/2012/10/15/el-salto-de-felix-baumgartner-en-python/

La ecuación que gobierna la caída de Felix es:

$$\displaystyle m \frac{d^2 y}{d t^2} = -m g + D$$

Siendo

$$D = \frac{1}{2} \rho v^2 C_D A$$

donde

* $m$ es la masa de Félix y la tomaremos $m = 80~\text{kg}$,
* $\rho$ es la densidad del aire **y depende de la altura**,
* $v = |\dot{y}|$ es la velocidad,
* $C_D$ es el coeficiente de rozamiento, que tomaremos* $C_D = 0.4$, y
* $A$ es un área de referencia y tomaremos $A = 1~\text{m}^2$.

\* <small>Fuente: http://fisicadepelicula.blogspot.com.es/2012/10/la-fisica-del-salto-baumgartner.html</small>

Además, necesitaremos la altura inicial $h_0 = 39000~\text{m}$.

In [1]:
import numpy as np

In [2]:
import matplotlib.pyplot as plt

## Atmósfera estándar

Necesitamos escribir funciones que nos den las condiciones en la atmósfera estándar.

### Temperatura

$$T(h) = \begin{cases} T_0 + \lambda h & 0 <= h <= 11000 \\ T(11000) & 11000 < h \end{cases}
\\ ~\\ T_0 = 288.16 K \\
\lambda = -6.5 \cdot 10^{-3}~\text{K/m}$$

In [3]:
# aeropython: preserve

def T_ISA(h):
    """Temperatura en función de la altitud según modelo ISA.

    Argumentos
    ----------
    h : Altura en metros.

    Devuelve
    --------
    T : Temperatura en Kelvin.

    """
    T0 = 288.16  # K
    ll = -6.5e-3  # K / m
    if 0 <= h <= 11000:
        T = T0 + ll * h
    elif 11000 < h:
        T = T0 + ll * 11000
    # Es preferible que la función tenga solo un `return`
    return T

In [4]:
help(T_ISA)

Help on function T_ISA in module __main__:

T_ISA(h)
    Temperatura en función de la altitud según modelo ISA.
    
    Argumentos
    ----------
    h : Altura en metros.
    
    Devuelve
    --------
    T : Temperatura en Kelvin.



Si quieres comprobar que tus funciones hacen lo que deben, puedes ejecutar estos tests:

In [5]:
from numpy.testing import assert_almost_equal

assert_almost_equal(T_ISA(0), 288.16)
assert_almost_equal(T_ISA(11000), 216.66)

<div class="alert alert-warning">**¡Importante!** Si utilizas condicionales para comprobar las capas de la atmósfera, seguramente tus funciones fallarán si las quieres representar utilizando un `linspace`. Para estos casos es mejor utilizar la función `np.select`:</div>

In [6]:
# aeropython: preserve
def T_ISA(h):
    """Temperatura en función de la altitud según modelo ISA.

    Argumentos
    ----------
    h : Altura en metros.

    Devuelve
    --------
    T : Temperatura en Kelvin.

    """
    # Con esta línea convertimos la entrada a un array
    h = np.asarray(h)

    T0 = 288.16  # K
    ll = -6.5e-3  # K / m

    T1 = T0 + ll * h
    T2 = T0 + ll * 11000

    # 0 <= h <= 110000 no funciona para arrays
    T = np.select([(0 <= h) & (h <= 11000), 11000 < h], [T1, T2])
    return T

In [7]:
# aeropython: preserve
T_ISA(0)

array(288.16)

In [8]:
# aeropython: preserve
T_ISA(np.array([0, 11000, 20000]))

array([288.16, 216.66, 216.66])

In [9]:
# aeropython: preserve
# Como hemos puesto `h = np.asarray(h)`, podemos hacer esto
T_ISA([0, 11000, 20000])

array([288.16, 216.66, 216.66])

Otra forma alternativa (idea de Alfredo y Laura):

In [23]:
# aeropython: preserve
def T_ISA(h):
    """Temperatura en función de la altitud según modelo ISA.

    Argumentos
    ----------
    h : Altura en metros.

    Devuelve
    --------
    T : Temperatura en Kelvin.

    """
    # Con esta línea convertimos la entrada a un array
    h = np.atleast_1d(h)

    T0 = 288.16  # K
    ll = -6.5e-3  # K / m

    T = np.zeros_like(h, dtype=float)

    T[(0 <= h) & (h <= 11000)] = T0 + ll * h[(0 <= h) & (h <= 11000)]
    T[11000 < h] = T0 + ll * 11000

    return T

In [24]:
# aeropython: preserve
T_ISA(0)

array([288.16])

In [25]:
# aeropython: preserve
T_ISA([0, 11000, 20000])

array([288.16, 216.66, 216.66])

### Densidad

$$ \rho(h) = \begin{cases} \rho_0 \left( \frac{T}{T_0} \right)^{-\frac{g}{\lambda R} - 1} & 0 <= h <= 11000 \\ \rho(11000)~e^{\frac{-g(z - 11000)}{R T}} & 11000 < h <= 20000 \end{cases} $$

$$\rho_0 = 1.225~\text{[SI]} \\
R = 287~\text{[SI]}$$

In [13]:
# aeropython: preserve
def rho_ISA(h):
    h = np.asarray(h)

    T0 = 288.16  # K
    ll = -6.5e-3  # K / m
    g = 9.8  # m / s2
    rho0 = 1.225  # kg / m3
    R = 287.0  # [SI]

    rho1 = rho0 * (T_ISA(h) / T0) ** (-g / (ll * R) - 1)
    
    rho2 = rho0 * (T_ISA(11000) / T0) ** (-g / (ll * R) - 1) * np.exp(-g * (h - 11000) / (R * T_ISA(h)))

    rho = np.select([(0 <= h) & (h <= 11000), 11000 < h], [rho1, rho2])
    return rho

In [14]:
# aeropython: preserve
rho_ISA([0, 11000, 20000])

array([1.225     , 0.36420497, 0.08817176])

## Ecuación diferencial

Recuerda de la clase 035 que `integrate.solve_ivp(f, (tini, tfin), y0)` resuelve ecuaciones del tipo:

$$ \frac{d\mathbf{y}}{dt}=\mathbf{f}(t, \mathbf{y})$$

con condiciones iniciales $\mathbf{y}(\mathbf{0}) = \mathbf{y_0}$. Por tanto, y al tratarse esta de una ecuación en derivada segunda, hay que hacer una reducción de orden.

Importamos el modulo necesario de scipy para poder integrar:

In [15]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt

In [16]:
# Función del sistema
def f(t, y):
    
    g = 9.8  # m / s2
    C_D = 0.4
    A = 1.0  # m^2
    m = 80  # kg
    return np.array([
        y[1],
        -g + rho_ISA(y[0]) * y[1] ** 2 * C_D * A / (2 * m)
    ])

# Condiciones iniciales
y0 = np.array([39000, 0])

# Vector de tiempos
t = np.linspace(0, 250)

# Integramos la ecuación
sol = integrate.solve_ivp(f, (0., 250.), y0)



In [17]:
sol

  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 200
     njev: 0
      nlu: 0
      sol: None
   status: 0
  success: True
        t: array([0.00000000e+00, 1.02040814e-04, 1.12244895e-03, 1.13265303e-02,
       1.13367344e-01, 1.13377548e+00, 1.13378569e+01, 3.20426377e+01,
       5.32000786e+01, 6.71939363e+01, 7.88156765e+01, 9.20905916e+01,
       1.04091133e+02, 1.16375357e+02, 1.28659581e+02, 1.39137985e+02,
       1.49795920e+02, 1.62178467e+02, 1.75166428e+02, 1.86518179e+02,
       1.98140366e+02, 2.09808405e+02, 2.20122600e+02, 2.30651899e+02,
       2.41359447e+02, 2.50000000e+02])
 t_events: None
        y: array([[ 3.90000000e+04,  3.90000000e+04,  3.90000000e+04,
         3.89999994e+04,  3.89999370e+04,  3.89937015e+04,
         3.83716328e+04,  3.40931351e+04,  2.66038371e+04,
         2.16669045e+04,  1.84853808e+04,  1.57871804e+04,
         1.38880010e+04,  1.22626956e+04,  1.08525792e+04,
         9.76950272e+03,  8.74652

In [27]:
sol.y

array([[ 3.90000000e+04,  3.90000000e+04,  3.90000000e+04,
         3.89999994e+04,  3.89999370e+04,  3.89937015e+04,
         3.83716328e+04,  3.40931351e+04,  2.66038371e+04,
         2.16669045e+04,  1.84853808e+04,  1.57871804e+04,
         1.38880010e+04,  1.22626956e+04,  1.08525792e+04,
         9.76950272e+03,  8.74652633e+03,  7.63751362e+03,
         6.55044802e+03,  5.65433011e+03,  4.78244381e+03,
         3.94793686e+03,  3.24048558e+03,  2.54491674e+03,
         1.86268210e+03,  1.32892448e+03],
       [ 0.00000000e+00, -9.99999974e-04, -1.09999997e-02,
        -1.10999997e-01, -1.11099946e+00, -1.11104846e+01,
        -1.10567569e+02, -2.96318286e+02, -3.79331910e+02,
        -3.13103337e+02, -2.36334093e+02, -1.75755913e+02,
        -1.43611154e+02, -1.22639284e+02, -1.07961633e+02,
        -9.94403608e+01, -9.28405690e+01, -8.65963824e+01,
        -8.11350813e+01, -7.69766647e+01, -7.32720167e+01,
        -6.99939690e+01, -6.73498706e+01, -6.49192790e+01,
        -6.26

In [19]:
sol.y.shape

(2, 26)

In [28]:
pos = sol.y[0, :]  # Primera fila: posición
vel = sol.y[1, :]   # Segunda fila: velocidad
t = sol.t              # tiempo

# Representamos la solución
fig, axes = plt.subplots(2, 1, sharex=True, figsize=(6, 6))
line, = axes[0].plot(t, pos / 1e3, label="Position $y$")
axes[0].set_ylabel("Position (km)")
axes[1].plot(t, vel, '--', color=line.get_color(), label="Velocity $\dot{y}$")
axes[1].set_ylabel("Velocity (m / s)")
axes[1].set_xlabel("Time (s)")
axes[0].legend()
axes[1].legend()
axes[0].set_title("Felix Baumgartner free fall")
fig.tight_layout()

## Barrera del sonido

La velocidad del sonido en el aire variará también, y lo hará de esta forma:

$$M = \frac{v}{c}$$

siendo

$$c = \sqrt{\gamma R T}$$

Como paso final, representa $M$ en función de $t$ y en la misma gráfica incluye una línea horizontal de trazo discontinuo donde $M = 1$.

Text(0.5,1,'Mach number')

In [21]:
gamma = 1.4
R = 287.0  # [SI]
c = np.sqrt(gamma * R * T_ISA(pos))

M = np.abs(vel) / c

plt.plot(t, M)
plt.plot(t, np.ones_like(t), 'k--')
plt.ylabel('Mach number')
plt.xlabel('Time (s)')
plt.title("Mach number")

**Barrera del sonido superada** ;)

--- 

__Referencias__

Material adaptado del "Curso AeroPython". AeroPython. https://github.com/AeroPython/Curso_AeroPython<br>

 <a rel="license" href="http://creativecommons.org/licenses/by/4.0/deed.es"><img alt="Licencia Creative Commons" style="border-width:0" src="http://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Curso AeroPython</span> por <span xmlns:cc="http://creativecommons.org/ns#" property="cc:attributionName">Juan Luis Cano Rodriguez y Alejandro Sáez Mollejo</span> se distribuye bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/deed.es">Licencia Creative Commons Atribución 4.0 Internacional</a>.

---
_Las siguientes celdas contienen configuración del Notebook_

_Para visualizar y utlizar los enlaces a Twitter el notebook debe ejecutarse como [seguro](http://ipython.org/ipython-doc/dev/notebook/security.html)_

    File > Trusted Notebook

In [2]:
# preserve
# Esta celda da el estilo al notebook
from IPython.core.display import HTML
css_file = '../styles/style.css'
HTML(open(css_file, "r").read())